# BASIC IDEA OF THE KERNEL

The data consists of a one dimensional time series x with 600 Mio data points. 

At test time, we will see a time series of length 150'000 to predict the next earthquake.

The idea of this kernel is to randomly sample chunks of length 150'000 from x, 

derive some features and use them to update weights of a recurrent neural net with 150'000 / 1000 = 150 time steps. 

In [1]:
import numpy as np 
import pandas as pd
import os
from tqdm import tqdm

# Fix seeds
from numpy.random import seed
seed(639)
from tensorflow import set_random_seed
set_random_seed(5944)

In [3]:
# Import
#float_data = pd.read_csv("../input/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values
df_train = pd.read_hdf("../input/train.hdf", key='0')
float_data = df_train.values

In [4]:
# Helper function for the data generator. Extracts mean, standard deviation, and quantiles per time step.
# Can easily be extended. Expects a two dimensional array.
def extract_features(z):
     return np.c_[z.mean(axis=1), 
                  z.min(axis=1),
                  z.max(axis=1),
                  z.std(axis=1)]

In [5]:
# For a given ending position "last_index", we split the last 150'000 values 
# of "x" into 150 pieces of length 1000 each. So n_steps * step_length should equal 150'000.
# From each piece, a set features are extracted. This results in a feature matrix 
# of dimension (150 time steps x features).  
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
       
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]

In [6]:
# Query "create_X" to figure out the number of features
X = create_X(float_data[0:150000])
n_features = X.shape[1]
print("Our RNN is based on %i features"% n_features)

Our RNN is based on 12 features


In [7]:
X.shape

(150, 12)

In [8]:
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1
     
    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        
        for j, row in enumerate(rows):
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            targets[j] = data[row - 1, 1]
        yield samples, targets

In [9]:
# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
#float_data[second_earthquake, 1]

In [10]:
# Initialize generators
batch_size = 32

#train_gen = generator(float_data, batch_size=batch_size) # Use this for better score
train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)

In [36]:
# Define model
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU, CuDNNLSTM, GRU, Bidirectional
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint, TensorBoard

In [40]:
cb = [
    ModelCheckpoint("model.hdf5", save_best_only=True, period=3),
    #TensorBoard(log_dir="tflog/", histogram_freq=1)
]

model = Sequential()
#model.add(CuDNNLSTM(48, input_shape=(None, n_features)))
model.add(CuDNNLSTM(48, input_shape=(None, n_features)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [41]:
# Compile and fit model
model.compile(optimizer=adam(lr=0.0005), loss="mae")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_10 (CuDNNLSTM)    (None, 48)                11904     
_________________________________________________________________
dense_23 (Dense)             (None, 10)                490       
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 12,405
Trainable params: 12,405
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=1,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

Epoch 1/30
1000/1000 [==============================] - 35s 35ms/step - loss: 2.3977 - val_loss: 1.8171
Epoch 2/30
1000/1000 [==============================] - 38s 38ms/step - loss: 2.1677 - val_loss: 2.0083
Epoch 3/30
1000/1000 [==============================] - 36s 36ms/step - loss: 2.1506 - val_loss: 2.0260
Epoch 4/30
1000/1000 [==============================] - 35s 35ms/step - loss: 2.1149 - val_loss: 1.8496
Epoch 5/30
1000/1000 [==============================] - 36s 36ms/step - loss: 2.1175 - val_loss: 1.6625
Epoch 6/30
1000/1000 [==============================] - 36s 36ms/step - loss: 2.1117 - val_loss: 1.9814
Epoch 7/30
1000/1000 [==============================] - 37s 37ms/step - loss: 2.0919 - val_loss: 1.8502
Epoch 8/30
1000/1000 [==============================] - 37s 37ms/step - loss: 2.0906 - val_loss: 1.8285
Epoch 9/30
1000/1000 [==============================] - 37s 37ms/step - loss: 2.0940 - val_loss: 1.9028
Epoch 10/30
1000/1000 [==============================] - 38s 38m

In [30]:
# Visualize accuracies
import matplotlib.pyplot as plt

def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

<Figure size 640x480 with 1 Axes>

In [ ]:
# Load submission file
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

# Load each test data, create the feature matrix, get numeric prediction
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

# Save
submission.to_csv('submission.csv')